## PART 1

In [13]:
with open('input.txt') as f:
    input = list(map(lambda x: x.strip(), f.readlines()))

max_z = -1
for i,l in enumerate(input):
    input[i] = [i+1] + list(map(lambda x: list(map(lambda y: int(y), x.split(','))), l.split('~')))
    if input[i][1][2] > max_z: max_z = input[i][1][2]
    if input[i][2][2] > max_z: max_z = input[i][2][2]


In [14]:
# input

In [15]:
Z_LAYERS = [[] for _ in range(max_z+1)]

In [16]:
for l in input:
    for z in range(l[1][2], l[2][2]+1):
        Z_LAYERS[z].append(tuple([l[0], tuple([l[1][0],l[2][0]]), tuple([l[1][1],l[2][1]])]))
        
# Z_LAYERS[::-1]

In [17]:
def compatible(c1, c2):
    if ((c1[1][0] > c2[1][1] or c2[1][0] > c1[1][1]) or 
        (c1[2][0] > c2[2][1] or c2[2][0] > c1[2][1])):
        return True
    else:
        return False

In [18]:
while True:

    changed = False

    for z in range(2, max_z+1):

        indexes_to_remove = []
        for i,cube in enumerate(Z_LAYERS[z]): 
            if all(compatible(cube,other_cube) for other_cube in Z_LAYERS[z-1]):
                Z_LAYERS[z-1].append(cube)
                indexes_to_remove.append(i)
                changed = True
        
        Z_LAYERS[z] = [x for i,x in enumerate(Z_LAYERS[z]) if i not in indexes_to_remove]
                
    if not changed: 
        break

# Z_LAYERS[::-1]

In [19]:
SUP_BY = dict()

for z in range(1, max_z+1):

    for i,cube in enumerate(Z_LAYERS[z]):
        for other_cube in Z_LAYERS[z-1]:
            if cube[0] == other_cube[0]: 
                continue
            if not compatible(cube,other_cube):
                if cube[0] not in SUP_BY:
                    SUP_BY[cube[0]] = []
                SUP_BY[cube[0]].append(other_cube[0])

# SUP_BY

In [20]:
res = set()
for v in SUP_BY.values():
    if len(v) == 1:
        res.add(v[0])

len(input)  - len(res)

522

## PART 2

da SUP_BY costruisco un grafo dove i -> j se il blocco i sostiene il blocco j

aggiungo un blocco fittizio 0 ad altezza 0 che sostiene tutti i blocchi che non hanno archi entranti

per ogni i, quando tolgo tutti i suoi archi uscenti, vedo quanti nodi del grafo smettono di essere raggiungibili da 0

i nodi che smettono di essere raggiungibili sono quelli che si muoverebbero se i venisse rimosso 

In [21]:
adj_list = [[] for _ in range(len(input)+1)]

for k,v in SUP_BY.items():
    for x in v:
        adj_list[x].append(k)

no_incoming_arcs = set(range(1,len(input)+1)) 
for row in adj_list:
    for el in row:
        no_incoming_arcs.discard(el)

adj_list[0] = list(no_incoming_arcs)

In [22]:
# adj_list  

In [23]:
import networkx as nx

G = nx.DiGraph()
for i in range(len(adj_list)):
    for j in adj_list[i]:
        G.add_edge(i, j)
        
# nx.draw(G, with_labels=True)

In [24]:
all_nodes = set(range(1,len(input)+1))

res = 0
for i in range(1,len(adj_list)):

    for j in adj_list[i]: 
        G.remove_edge(i, j)

    res += len(input) - len(nx.descendants(G, 0))

    for j in adj_list[i]: 
        G.add_edge(i, j)

res

83519